# Attention
Class = 2.

Integrate (1) Nucleus features as recored by CellProfiler Proces100
and (2) attention level recorded by the CNN.
Use Random Forest to say what CP features are predictive of attention.
Here, we do the analysis per class, just because that is easiest.

In [1]:
import datetime
import numpy as np
import pandas as pd
from RandomForestUtil import RF_Util
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
ATTFILEPATH='/home/jrm/Adjeroh/Naved/CP_80K/attention_nucleus/' # Alien
ATTFILENAME='class2_avg_temp.csv'
NUCFILEPATH='/home/jrm/Adjeroh/Naved/CP_80K/Output2/'
NUCFILENAME='Process100_Nucleus.csv'

## Load labels
The labels are the temperatures of the CNN attention heatmap,
specifically an average across the bounding box of each nucleus.

In [3]:
att_df=pd.read_csv(ATTFILEPATH+ATTFILENAME)
cols={'ImageNumber':'PatchNumber',' ObjectNumber':'ObjectNumber',' AvgTemp':'AvgTemp'}
att_df.rename(columns=cols,inplace=True)
att_df.sort_values(by=['PatchNumber','ObjectNumber'],axis=0,inplace=True)
att_df.reset_index(drop=True,inplace=True)

In [4]:
# Later, try the RandomForestRegressor. For now use bins and classification.
bins = [0, 0.3, 0.45, 0.6, 0.75, 1.0]
labels = ['to30%', 'to45%', 'to60%', 'to75%', 'to100%']
att_df['bin'] = pd.cut(att_df['AvgTemp'], bins=bins, labels=labels)
att_df

,PatchNumber,ObjectNumber,AvgTemp,bin
0,1,1,0.177721,to30%
1,1,2,0.247854,to30%
2,1,3,0.718962,to75%
3,1,4,0.475308,to60%
4,1,5,0.194604,to30%
...,...,...,...,...
253549,13950,3,0.731186,to75%
253550,13950,4,0.864379,to100%
253551,13950,5,0.558921,to60%
253552,13950,6,0.585450,to60%


In [5]:
ytrain = att_df['bin'] 
ytrain.value_counts()

to30%     55799
to100%    55749
to60%     52315
to45%     48517
to75%     41174
Name: bin, dtype: int64

## Load features
These are features of nuclei as determined by our "Process100" pipeline for CellProfiler.
These are the nuclei used to set the bounding boxes for the temperature lables.
Other than that, the features and temperatures were each derived independently.

In [6]:
nuc_df=pd.read_csv(NUCFILEPATH+NUCFILENAME)
cols={'ImageNumber':'PatchNumber'}
nuc_df.rename(columns=cols,inplace=True)
nuc_df.sort_values(by=['PatchNumber','ObjectNumber'],axis=0,inplace=True)
nuc_df.reset_index(drop=True,inplace=True)

In [7]:
Xtrain = nuc_df.drop(columns=['PatchNumber','ObjectNumber'])
Xtrain

,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_BoundingBoxMaximum_X,AreaShape_BoundingBoxMaximum_Y,AreaShape_BoundingBoxMinimum_X,AreaShape_BoundingBoxMinimum_Y,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,...,Texture_Variance_Hematoxylin_4_02_256,Texture_Variance_Hematoxylin_4_03_256,Texture_Variance_Hematoxylin_5_00_256,Texture_Variance_Hematoxylin_5_01_256,Texture_Variance_Hematoxylin_5_02_256,Texture_Variance_Hematoxylin_5_03_256,Texture_Variance_Hematoxylin_7_00_256,Texture_Variance_Hematoxylin_7_01_256,Texture_Variance_Hematoxylin_7_02_256,Texture_Variance_Hematoxylin_7_03_256
0,416,500,139,20,114,0,126.122596,8.918269,416.0,-2.131628e-14,...,809.352884,775.851067,817.607072,739.400054,832.760417,809.951419,860.392197,804.253719,900.644998,889.351473
1,886,1312,300,32,259,0,279.781038,14.164786,886.0,-1.278977e-13,...,1430.767357,1350.777242,1252.271612,1386.291754,1437.280463,1408.422304,1285.271855,1426.075016,1396.052168,1481.726400
2,463,594,300,74,278,47,290.241901,58.807775,463.0,6.394885e-13,...,510.877864,462.652829,503.335381,500.912176,501.229896,448.095553,487.812999,528.940983,488.124195,419.667092
3,1550,2430,233,88,188,34,206.715484,59.100000,1550.0,1.271871e-12,...,1209.931031,1219.549334,1194.106218,1194.917997,1213.345428,1230.076802,1179.198748,1131.325432,1210.838160,1243.478218
4,542,696,95,93,66,69,77.942804,79.778598,542.0,1.563194e-13,...,1156.289785,1010.067218,1066.181671,1132.622638,1177.827161,1023.289737,1147.856933,1222.939834,1233.017089,1101.174276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253549,740,1088,40,146,6,114,21.829730,129.856757,740.0,-9.237056e-14,...,984.281680,1017.922243,966.109935,965.942061,989.470614,1021.824556,953.574868,958.618873,1005.622440,1035.074711
253550,371,480,300,162,284,132,292.601078,146.099730,371.0,2.611245e-13,...,842.140864,834.593758,829.197083,825.700484,850.333404,889.873450,849.192536,847.952130,876.377961,955.533265
253551,185,255,108,276,91,261,99.367568,267.713514,185.0,1.243450e-13,...,1307.065350,1337.862422,1260.476304,1243.953731,1279.421766,1535.064563,1288.891823,1191.284894,1296.442899,1957.384400
253552,198,340,124,277,104,260,114.207071,267.404040,198.0,-1.367795e-13,...,955.581027,939.107175,979.309434,976.959562,914.675850,1036.393869,955.618395,995.011696,946.081956,1311.715780


In [8]:
att_df=None
nuc_df=None
print('Count Nan before:',Xtrain.isna().sum().sum())
Xtrain.fillna(0,inplace=True)
print('Count Nan after:',Xtrain.isna().sum().sum())

Count Nan before: 761312
Count Nan after: 0


## Measure agreement (with a random forest classifier)
Can the RF predict the temperature of each nucleus given only the CellProfiler features?

In [9]:
print(datetime.datetime.now())
print("Shuffle...")
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-09 08:21:55.432783
Shuffle...


In [10]:
print(datetime.datetime.now())
print('Cross validation...')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('Accuracy mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-09 08:21:56.015568
Cross validation...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 7.5min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.5min remaining:    0.0s


[CV] END .................................................... total time= 7.5min
[CV] END .................................................... total time= 7.5min
[CV] END .................................................... total time= 7.5min
[CV] END .................................................... total time= 7.4min
[0.93953974 0.94334563 0.94202441 0.94210329 0.93999211]
Accuracy mean 0.9414 +/- 0.0014


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 37.5min finished


## Measure feature importance

In [11]:
print(datetime.datetime.now())
print('Re-train on full training set...')
rf1 = None
rf2 = RF_Util()
rf2.set_train(Xtrain,ytrain)
rf2.fit()
print(datetime.datetime.now())
print('...and rank the features by importance.')
top = rf2.important_features()
pd.set_option('display.max_rows', None)
top.loc[:200]

2022-06-09 08:59:25.101817
Re-train on full training set...
2022-06-09 09:09:14.947338
...and rank the features by importance.


,0,1
0,0.075280,Location_Center_X
1,0.074640,Location_CenterMassIntensity_X_Hematoxylin
2,0.069417,AreaShape_BoundingBoxMaximum_X
3,0.069109,AreaShape_Center_X
4,0.060710,AreaShape_BoundingBoxMinimum_X
5,0.056438,Location_MaxIntensity_X_Hematoxylin
6,0.041949,Location_Center_Y
7,0.041433,AreaShape_Center_Y
8,0.040465,Location_CenterMassIntensity_Y_Hematoxylin
9,0.036782,AreaShape_BoundingBoxMaximum_Y
